In [0]:
!pip install pycantonese

    100% |████████████████████████████████| 686kB 9.3MB/s 


In [0]:
import pycantonese as pc
import pdb
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import itertools
import pandas as pd
import numpy as np
import pickle
import os

from google.colab import drive
drive.mount('/content/gdrive')

model_dir = '/content/gdrive/My Drive/canto_rnn_model/v2/'
# import nltk
# from nltk.tag import hmm

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Corpus Preprocessing Helper Functions

In [0]:
def stage1_preprocessing(sent,gen_y=True):
    res_X = []
    res_Y = []
    for token_tup in sent:
        token = token_tup[0]
        token_words_X = [t for t in token]
        res_X += token_words_X
        if gen_y:
            token_words_Y = [0] * len(token)
            token_words_Y[-1] = 1
            res_Y += token_words_Y
    return [res_X,res_Y]

batch_stage1_preprocessing = lambda sents,gen_y: [stage1_preprocessing(sent,gen_y) for sent in sents]

def stage2_build_dictionary(all_characters_with_dup):
    all_characters = pd.Series([i for i in set(all_characters_with_dup)]).sort_values().reset_index(drop=True)
    ch2id = pd.Series({ch:ind for ind,ch in all_characters.to_dict().items()})+1
    ch2id['<UNKNOWN>'] = ch2id.max() + 1
    ch2id['<PAD>'] = ch2id.max() + 1
    ch2id['<SOS>'] = ch2id.max() + 1
    ch2id['<EOS>'] = ch2id.max() + 1
    id2ch = pd.Series({_id:ch for ch,_id in ch2id.to_dict().items()})
    return ch2id,id2ch

def stage3_preprocessing(ch2id,stage1_sent_X):
    res = pd.Series({
        'len':len(stage1_sent_X),
        'X':str([ch2id.get(ch) if ch2id.get(ch) is not None else ch2id.get('<UNKNOWN>') for ch in stage1_sent_X])
    })
    return res

batch_stage3_preprocessing = lambda ch2id,sents_X: [stage3_preprocessing(ch2id,sent) for sent in sents_X]

def stage3_padding(stage3_sent_in_str,pad_with,max_len):
    stage3_sent = eval(stage3_sent_in_str)
    res = stage3_sent.copy()
    res += [pad_with] * (max_len - len(stage3_sent))
    return str(res)


def stage4_split_X(row_str):
    row = eval(row_str)
#     x = pd.Series(['X_{}'.format(str(ind)):r[0] for ind,r in enumerate(row)])

def batch_stage4_split(stage3_df):
    len_df = stage3_df[['len']]
    xy = stage3_df.xy
    xy_df = xy.apply(lambda row: pd.Series(eval(row)) )
    print(xy_df)
    

########################batch generator
class TrainingBatchGenerator:
    def __init__(self,training_data,batch_size):
        self.training_data = training_data
        self.head = 0
        self.batch_size = batch_size
    
    def next_batch(self):
        batch = self.training_data[self.head : self.head + self.batch_size ]
        self.head += self.batch_size
        if self.head >= len(self.training_data):
            self.head = 0
        return batch    

# Preprocess Corpus

In [0]:
train_size = 0.7
corpus = pc.hkcancor()
corp_sents = corpus.tagged_sents()
corp_sents = shuffle(corp_sents,random_state=0)
t_sents,v_sents = train_test_split(corp_sents,random_state=0,train_size=train_size)

####stage 1
t1_sents = np.array(batch_stage1_preprocessing(t_sents,gen_y=True))
t1_sents_X,t1_sents_Y = t1_sents[:,0],t1_sents[:,1]
v1_sents = np.array(batch_stage1_preprocessing(v_sents,gen_y=True))
v1_sents_X,v1_sents_Y = v1_sents[:,0],v1_sents[:,1]


####stage 2: build character dictionary
all_characters_with_dup = list(itertools.chain.from_iterable(t1_sents_X)) + list(itertools.chain.from_iterable(v1_sents_X))
ch2id,id2ch = stage2_build_dictionary(all_characters_with_dup)

####stage 3: convert character to id
t3_sents = pd.DataFrame(batch_stage3_preprocessing(ch2id,t1_sents_X))
t3_sents['Y'] = [str(Y) for Y in t1_sents_Y]
v3_sents = pd.DataFrame(batch_stage3_preprocessing(ch2id,v1_sents_X))
v3_sents['Y'] = [str(Y) for Y in v1_sents_Y]
all_sents = pd.concat([t3_sents,v3_sents]).reset_index(drop=True)
max_len = all_sents.len.max()

###stage 3.5: pad sentences so that they are all of the same length
t3_sents.X = t3_sents.X.apply(stage3_padding,pad_with=ch2id.get('<PAD>'),max_len=max_len)
t3_sents.Y = t3_sents.Y.apply(stage3_padding,pad_with=1,max_len=max_len)
v3_sents.X = v3_sents.X.apply(stage3_padding,pad_with=ch2id.get('<PAD>'),max_len=max_len)
v3_sents.Y = v3_sents.Y.apply(stage3_padding,pad_with=1,max_len=max_len)


###stage 4: separate len,X,Y
ohe = OneHotEncoder(sparse=False,categories=[[0,1]]*max_len)
t4_sents_len = t3_sents.len
t4_sents_mask = t3_sents.len.apply(lambda row:pd.Series([1.] * row + [0.]*(max_len - row)))
t4_sents_mask = t4_sents_mask.values.reshape((len(t4_sents_len),max_len))
t4_sents_X = t3_sents.X.apply(lambda row: pd.Series(eval(row)))
t4_sents_Y = t3_sents.Y.apply(lambda row: pd.Series(eval(row)))
t4_sents_Y_onehot = ohe.fit_transform(t4_sents_Y).reshape((len(t4_sents_Y),max_len,2))

v4_sents_len = v3_sents.len
v4_sents_mask = v3_sents.len.apply(lambda row:pd.Series([1.] * row + [0.]*(max_len - row)))
v4_sents_mask = v4_sents_mask.values.reshape((len(v4_sents_len),max_len))
v4_sents_X = v3_sents.X.apply(lambda row: pd.Series(eval(row)))
v4_sents_Y = v3_sents.Y.apply(lambda row: pd.Series(eval(row)))
v4_sents_Y_onehot = ohe.transform(v4_sents_Y).reshape((len(v4_sents_Y),max_len,2))


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [0]:
data2besaved = {
    'ohe':ohe,
    't4_sents_len':t4_sents_len,
    't4_sents_mask':t4_sents_mask,
    't4_sents_mask':t4_sents_mask,
    't4_sents_X':t4_sents_X,
    't4_sents_Y':t4_sents_Y,
    't4_sents_Y_onehot':t4_sents_Y_onehot,
    'v4_sents_len':v4_sents_len,
    'v4_sents_mask':v4_sents_mask,
    'v4_sents_mask':v4_sents_mask,
    'v4_sents_X':v4_sents_X,
    'v4_sents_Y':v4_sents_Y,
    'v4_sents_Y_onehot':v4_sents_Y_onehot,
    'ch2id':ch2id,
    'id2ch':id2ch,
}

with open(os.path.join(model_dir,'data2besaved.pkl'),'wb') as f:
  pickle.dump(data2besaved,f)

In [0]:
with open(os.path.join(model_dir,'data2besaved.pkl'),'rb') as f:
  data2besaved = pickle.load(f)
  ohe = data2besaved['ohe']
  t4_sents_len = data2besaved['t4_sents_len']
  t4_sents_mask = data2besaved['t4_sents_mask']
  t4_sents_mask = data2besaved['t4_sents_mask']
  t4_sents_X = data2besaved['t4_sents_X']
  t4_sents_Y = data2besaved['t4_sents_Y']
  t4_sents_Y_onehot = data2besaved['t4_sents_Y_onehot']
  v4_sents_len = data2besaved['v4_sents_len']
  v4_sents_mask = data2besaved['v4_sents_mask']
  v4_sents_mask = data2besaved['v4_sents_mask']
  v4_sents_X = data2besaved['v4_sents_X']
  v4_sents_Y = data2besaved['v4_sents_Y']
  v4_sents_Y_onehot = data2besaved['v4_sents_Y_onehot']
  ch2id = data2besaved['ch2id']
  id2ch = data2besaved['id2ch']
  max_len = t4_sents_X.shape[1]

# Define Many2Many RNN graph

In [0]:
####################wrapp rnn as estimator
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices( (features, labels) )
    dataset = dataset.repeat().batch(batch_size)#.shuffle(1000).repeat().batch(batch_size)
    return dataset


def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
#     features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = (features,features)
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

def my_model(features, labels, mode, params):
    
    vocabulary_size = params.get('vocabulary_size')
    embedding_size = params.get('embedding_size')
    num_sampled = params.get('num_sampled')
    rnn_cell_sizes = params.get('rnn_cell_sizes')
    learning_rate = params.get('learning_rate')
    seed = params.get('seed')
    n_classes = params.get('n_classes')
    max_len = params.get('max_len')
#     log_step_size = params.get('log_step_size')
#     num_epochs = params.get('num_epochs')

    rnn_cell_sizes += [n_classes]    
    
    ########Placeholders for inputs
    train_inputs = tf.cast(features[0], tf.int32)
    if labels is not None:
        train_labels = tf.cast(labels, tf.float32)
        
    train_masks = tf.cast(features[1],tf.float32)

    #######embedding layer
    embeddings = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size], -0.1, 0.1),\
                             name='embeddings',dtype=tf.float32)
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    #######extend to all timestep
    rnn_layers = [tf.nn.rnn_cell.LSTMCell(size) for size in rnn_cell_sizes ]
    multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)
    
    bw_rnn_layers = [tf.nn.rnn_cell.LSTMCell(size) for size in rnn_cell_sizes ]
    bw_multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(bw_rnn_layers)    
    
    ######forward RNN
#     outputs,state = tf.nn.dynamic_rnn(cell=multi_rnn_cell,\
#                                       inputs=embed,\
#     #                                        sequence_length=train_len,\
#                                        dtype=tf.float32)

    #####bidirectional RNN
    bidirectional_outputs,state = tf.nn.bidirectional_dynamic_rnn(cell_fw=multi_rnn_cell,\
                                                                  cell_bw=bw_multi_rnn_cell,\
                                                                  inputs=embed,\
#                                                                   sequence_length=train_len,\
                                                                  dtype=tf.float32)
    outputs = tf.concat([bidirectional_outputs[0], bidirectional_outputs[1]], 2)
    
    processed_outputs = tf.transpose(outputs,perm=[1,0,2])
    processed_masks = tf.transpose(train_masks,perm=[1,0])
    pdb.set_trace()
    if labels is not None:
        processed_labels = tf.transpose(train_labels,perm=[1,0,2])
        scan_tuples = (processed_outputs,processed_labels,processed_masks)#,processed_masks_non_onehot)
    

    # #####output_at_last_timestep_weight
    W_init_val = np.random.uniform(-1,0,(rnn_cell_sizes[-1] * 2, n_classes))
    W_out = tf.Variable(tf.truncated_normal([rnn_cell_sizes[-1] * 2, n_classes], -0.1, 0.1),\
                        name='W_out',dtype=tf.float32)

    b_init_val = np.random.uniform(-1,0,(n_classes,))
    b_out = tf.Variable(tf.truncated_normal([n_classes,], -0.1, 0.1),\
                        name='b_out',dtype=tf.float32)
    
    
    def pred_prob(_,current_step_output):
        logits = tf.matmul(current_step_output,W_out) + b_out
        probs = tf.nn.softmax(logits)
        return probs
    
    pred_probs_initializer = tf.zeros(tf.shape(processed_outputs))
    pred_probs_initializer_sample = pred_probs_initializer[0,:,:2]
    
    pred_probs = tf.transpose( tf.scan(pred_prob,processed_outputs,initializer=pred_probs_initializer_sample) ,perm=[1,0,2] )    
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        prediction_bundle = {
            'probabilities': pred_probs,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=prediction_bundle)    

    # Define loss and optimizer
    # weights = tf.trainable_variables()
    # l1_regularizer = tf.contrib.layers.l1_regularizer(
    #    scale=0.005, scope=None
    # )
    # regularization_penalty = tf.contrib.layers.apply_regularization(l1_regularizer, weights)
    def output_step_v2(_,current_tup):
        current_step_output,current_step_label,current_step_mask = current_tup#,current_step_mask_non_onehot = current_tup
        logits = tf.matmul(current_step_output,W_out) + b_out
        current_step_ce = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=current_step_label)
        current_step_loss = tf.reduce_sum(current_step_ce * current_step_mask)
        return current_step_loss#(current_step_loss,current_step_label,current_step_mask)
      
        
    loss_op = tf.reduce_sum(tf.scan(output_step_v2,scan_tuples,initializer=0.))
    
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode, loss=loss_op, eval_metric_ops=metrics)
    
    
    # Create training op.
    assert mode == tf.estimator.ModeKeys.TRAIN
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss_op, train_op=train_op)


class Many2ManyRNNEstimator:
    def __init__(self,params,model_dir):
        self.estimator = tf.estimator.Estimator(model_fn=my_model,model_dir=model_dir,params=params)

    def train(self,X_df,X_mask,y_oh,batch_size,steps):
        self.estimator.train(input_fn=lambda:train_input_fn((X_df,X_mask),y_oh,batch_size),steps=steps)
        print('finish training')

    def evaluate_performance(self,X_df,X_mask,y_non_onehot,sents_len,batch_size):
        my_pred_prob = np.array([ps['probabilities'] for ps in self.predict_probs(X_df,X_mask,batch_size)])
        my_pred_classes = np.argmax(my_pred_prob,axis=2)
        sents_len_df = pd.DataFrame({'len':sents_len}).reset_index()
        y_non_onehot = np.array(y_non_onehot)
        def find_rowise_accuracy(row,my_pred_classes,sents_Y):
            row_len = row.len
            ind = row.name
            my_pred = my_pred_classes[ind][:row_len].reshape(-1)
            true_pred = sents_Y[ind][:row_len].reshape(-1)
            
            _df = pd.DataFrame({'pred':my_pred,'true':true_pred})
            true_Y_count = _df.true.value_counts()
            true_Y_count =  true_Y_count.rename({ind:'total_{}_count'.format(str(ind)) for ind in true_Y_count.index})

            true_correct_count = _df[_df.pred == _df.true].true.value_counts()
            true_correct_count =  true_correct_count.rename({ind:'total_{}_correct_count'.format(str(ind)) for ind in true_correct_count.index})
            
            res = pd.concat([true_Y_count,true_correct_count])
#             pdb.set_trace()
            return res

        res = sents_len_df.apply(find_rowise_accuracy,axis=1,my_pred_classes=my_pred_classes,sents_Y=y_non_onehot)
        
        return res
    
    def predict(self,X_df,X_mask,batch_size):
        my_pred_prob = np.array([ps['probabilities'] for ps in self.predict_probs(X_df,X_mask,batch_size)])
        my_pred_classes = np.argmax(my_pred_prob,axis=2)
        return my_pred_classes
      

    def predict_probs(self,X_df,X_mask,batch_size):
        return self.estimator.predict(input_fn=lambda:eval_input_fn((X_df,X_mask),
                                                                    labels=None,
                                                                    batch_size=batch_size))

# Actual Training

In [0]:
params = {
    'seed' : 0,
    'vocabulary_size' : len(ch2id),
    'embedding_size' : 8,
    'num_sampled' : 20,
    'rnn_cell_sizes' : [4],
    'learning_rate' : 0.001,
    'seed' : 0,
    'n_classes' : 2,
    'max_len':max_len,
    'log_step_size':500,
    'num_epochs':30000
}

###stage 5: to batches
batch_size = 1024
X_batch_generator = TrainingBatchGenerator(t4_sents_X,batch_size)
Y_batch_generator = TrainingBatchGenerator(t4_sents_Y_onehot,batch_size)
mask_batch_generator = TrainingBatchGenerator(t4_sents_mask,batch_size)
len_batch_generator = TrainingBatchGenerator(t4_sents_len,batch_size)


rnn = Many2ManyRNNEstimator(params=params,model_dir=model_dir)
v_tmp_res = rnn.evaluate_performance(v4_sents_X,v4_sents_mask,v4_sents_Y,v4_sents_len,batch_size)
# t_tmp_res = rnn.evaluate_performance(t4_sents_X,t4_sents_mask,t4_sents_Y,t4_sents_len,batch_size)

# for i in range(10):
#   rnn.train(t4_sents_X,t4_sents_mask,t4_sents_Y_onehot,batch_size,steps=1000)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/content/gdrive/My Drive/canto_rnn_model/v2/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd12a7f9be0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
> <ipython-input-7-b01e6c71cc47>(75)my_model()
-> if labels is not None:
(Pdb) 

BdbQuit: ignored

In [0]:
v_tmp_res0 = v_tmp_res[['total_0_correct_count','total_0_count']].fillna(0).sum(axis=0)

print('0-rule acc = ', v_tmp_res0.total_0_correct_count / v_tmp_res0.total_0_count )

v_tmp_res1 = v_tmp_res[['total_1_correct_count','total_1_count']].fillna(0).sum(axis=0)

print('1-rule acc = ', v_tmp_res1.total_1_correct_count / v_tmp_res1.total_1_count )

print('overall-rule acc = ', (v_tmp_res1.total_1_correct_count + v_tmp_res0.total_0_correct_count) / (v_tmp_res1.total_1_count + v_tmp_res0.total_0_count) )

0-rule acc =  0.90605738575983
1-rule acc =  0.9751433197671874
overall-rule acc =  0.9588411321196315


In [0]:
v_tmp_res['0_acc'] = v_tmp_res.total_0_correct_count.fillna(0) / v_tmp_res.total_0_count.fillna(0)
print( v_tmp_res['0_acc'].mean() )
print( v_tmp_res['0_acc'][~v_tmp_res['0_acc'].isnull()].mean() )
# print( zero_acc.mean() )
# v_tmp_res.total_0_correct_count.sort_values(ascending=False)


0.9022781171156257
0.9022781171156257


In [0]:
t_tmp_res0 = t_tmp_res[['total_0_correct_count','total_0_count']].fillna(0).sum(axis=0)

print('0-rule acc = ', t_tmp_res0.total_0_correct_count / t_tmp_res0.total_0_count )

t_tmp_res1 = t_tmp_res[['total_1_correct_count','total_1_count']].fillna(0).sum(axis=0)

print('1-rule acc = ', t_tmp_res1.total_1_correct_count / t_tmp_res1.total_1_count )
print('overall-rule acc = ', (t_tmp_res1.total_1_correct_count + t_tmp_res0.total_0_correct_count) / (t_tmp_res1.total_1_count + t_tmp_res0.total_0_count) )

0-rule acc =  0.9582958109492904
1-rule acc =  0.9890563898577043
overall-rule acc =  0.9817877636078132


In [0]:
t_tmp_res['0_acc'] = t_tmp_res.total_0_correct_count.fillna(0) / t_tmp_res.total_0_count.fillna(0)
print( t_tmp_res['0_acc'].mean() )
print( t_tmp_res['0_acc'][~t_tmp_res['0_acc'].isnull()].mean() )
# print( zero_acc.mean() )
# v_tmp_res.total_0_correct_count.sort_values(ascending=False)


0.9526446108935582
0.9526446108935582


# Tokenization


In [0]:
train_size = 0.7
corpus = pc.hkcancor()
corp_sents = corpus.tagged_sents()
corp_sents = shuffle(corp_sents,random_state=0)
_,v_sents = train_test_split(corp_sents,random_state=0,train_size=train_size)


selected_sents = v_sents
# selected_sents
sent_batch = [''.join([j[0] for j in i]) for i in selected_sents]
s2sents_df = pd.DataFrame(batch_stage3_preprocessing(ch2id,sent_batch))
sents_X = s2sents_df.X.apply(stage3_padding,pad_with=ch2id.get('<PAD>'),max_len=max_len)
sents_len = s2sents_df.len
sents_mask = sents_len.apply(lambda row:pd.Series([1.] * row + [0.]*(max_len - row)))
sents_mask = sents_mask.values.reshape((len(sents_mask),max_len))
sents_X = sents_X.apply(lambda row: pd.Series(eval(row)))

rnn = Many2ManyRNNEstimator(params=params,model_dir=model_dir)
sents_prob = rnn.predict(sents_X,sents_mask,batch_size=1024)

for i in range(len(sent_batch)):
  sent_len = sents_len.values[i]
  sent_pred_class = sents_prob[i][:sent_len]
  sent = sent_batch[i]
  print('corpus: ',[j[0] for j in selected_sents[i]])
#   print(sent_pred_class)
  j = 0
  buffer = ''
  my_tokens = []
  while j < len(sent):
    ch = sent[j]
    tokenize_mask = sent_pred_class[j]
    buffer += ch
    if tokenize_mask == 1:
      my_tokens.append(buffer)
      buffer = ''
    j += 1
  print('mypred: ',my_tokens)
#   print([sent[j] for j in range(len(sent)) ])
  
  
#   print([str(sent_pred_class[j]) for j in range(len(sent)) ])
  print()
  
  

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/content/gdrive/My Drive/canto_rnn_model/v2/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f25d989bcf8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INF